# Rede Neural

In [174]:
# Imports
import random
import numpy as np

## Classe Network

In [175]:
class Network(object):

    def __init__(self, sizes):
        self.num_layers = len(sizes)   # número de camadas da rede
        self.sizes = sizes             # lista com o número de neurônios em cada camada
        
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] # lista de listas de bias
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] # lista de listas de pesos

    def training(self, training_data, method_auto, epochs, mini_batch_size, eta, test_data=None):
        training_data = list(training_data)
        n = len(training_data) # número de entradas de treinamento

        if test_data:
            test_data = list(test_data)
            n_test = len(test_data) # número de entradas de teste

        for j in range(epochs):
            random.shuffle(training_data) # embaralha os dados de treinamento
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)] # divide os dados de treinamento em mini lotes
            
            if method_auto == "sgd":
                for mini_batch in mini_batches:
                    self.update_sgd(mini_batch, eta)
            elif method_auto == "pso":
                for mini_batch in mini_batches:
                    self.update_pso(mini_batch)
                    break
            
            if test_data:
                print("Epoch {} : {} / {}".format(j,self.evaluate(test_data),n_test));
            else:
                print("Epoch {} finalizada".format(j))

### ATUALIZAÇÃO COM PARTICULE SWARM OPTMIZATION ###########################################
    def update_pso(self, mini_batch):
        nabla_b = []
        nabla_w = []
        for i in range(len(self.biases)):
            nabla_b.append(np.reshape(self.biases[i], len(self.biases[i])))
            print(self.biases[i])
        for i in range(len(self.weights)):
            nabla_w.append(np.reshape(self.weights[i], len(self.weights[i])))
            print(self.weights[i])
            
        from itertools import chain
        list_comb_bw = zip(self.biases, self.weights)
        #for particule in list_comb_bw:
            #print(list(chain(particule)))

        #for x, y in mini_batch:
            

### ATUALIZAÇÃO COM STOCHASTIC GRADIENT DESCENDENT ###########################################
    def update_sgd(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y) # calcula o gradiente
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)] 
        
        self.weights = [w-(eta/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        # Feedforward
        activation = x

        # Lista para armazenar todas as ativações, camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores z, camada por camada
        zs = [] 

        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        # Aqui, l = 1 significa a última camada de neurônios, l = 2 é a segunda e assim por diante. 
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)
#############################################################################################

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b) # ativação da camada com função sigmoid
        return a

In [176]:
from itertools import chain
l1 = ['a', 'b']
l2 = [1, 2]
list_comb_bw = zip(l1, l2)
for particule in list_comb_bw:
    print(list(chain(particule)))

['a', 1]
['b', 2]


In [177]:
# Função de Ativação Sigmóide
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

# Teste com Gradiente Descende e Backpropagation

In [178]:
# Imports
import mnist_loader

training_data, validation_data, test_data = mnist_loader.load_data_wrapper()
training_data = list(training_data)


net = Network([784, 30, 10])
#net.training(training_data,'sgd', 30, 10, 3.0, test_data=test_data)
net.training(training_data,'pso', 1, 10, 3.0, test_data=test_data)

[[-0.12106628]
 [-0.09913118]
 [-0.42268925]
 [-0.89802033]
 [-1.47384925]
 [ 2.07869423]
 [ 0.81005913]
 [-1.80473984]
 [-1.12750612]
 [ 1.33505805]
 [ 1.5284314 ]
 [ 0.63717277]
 [ 0.39174347]
 [ 1.40911934]
 [ 0.22917535]
 [-1.10110683]
 [-0.16241956]
 [-1.26930386]
 [-1.36507526]
 [ 1.3329589 ]
 [ 0.10898515]
 [-0.84309332]
 [ 0.18425875]
 [ 1.14042205]
 [ 1.07546823]
 [ 1.20407698]
 [-1.14209708]
 [ 1.0963799 ]
 [-0.0114067 ]
 [ 0.46167057]]
[[ 1.98642641]
 [-1.70375508]
 [-1.02074499]
 [-0.03764122]
 [-1.23145817]
 [-0.77822488]
 [-0.2969054 ]
 [ 2.92686212]
 [ 0.27768863]
 [-0.82734502]]


ValueError: cannot reshape array of size 23520 into shape (30,)